<center><img src="https://github.com/DACSS-CSSmeths/guidelines/blob/main/pics/small_logo_ccs_meths.jpg?raw=true" width="700"/></center>


<a target="_blank" href="https://colab.research.google.com/github/DACSS-CSSmeths-winter/optimization_AHP/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

_____
<a id='home'></a>


# Introduction to Optimization for Decision Making


# Multicriteria Decision-Making

## AHP in Python

### Load the Data & Libraries

#### Install and activate libraries needed

##### Let's verify

In [49]:
# Import pandas, numpy, and networkx

import pandas as pd
import numpy as np
import networkx as nx

#### Load the data from Github

In [50]:
# Read the excel sheet data from Github

linkAHP = 'https://github.com/DACSS-690C-Winter-2025-MG/Homework_3/raw/refs/heads/main/DACSS%20690C%20Homework%203%20AHP%20Excel%20Table.xlsx'

#### Load criterion sheets, verify data & cretae adjacency matrixes

In [51]:
# Load criterion sheets and create adjacency matrixes for each

pairwise_learning = pd.read_excel(linkAHP, sheet_name='Learning', index_col=0)
print("=== Learning ===")
print(pairwise_learning, "\n")

pairwise_friends = pd.read_excel(linkAHP, sheet_name='Friends', index_col=0)
print("=== Friends ===")
print(pairwise_friends, "\n")

pairwise_schoollife = pd.read_excel(linkAHP, sheet_name='School Life', index_col=0)
print("=== School Life ===")
print(pairwise_schoollife, "\n")

pairwise_vocational = pd.read_excel(linkAHP, sheet_name='Vocational Training', index_col=0)
print("=== Vocational Training ===")
print(pairwise_vocational, "\n")

pairwise_college = pd.read_excel(linkAHP, sheet_name='College Prep', index_col=0)
print("=== College Prep ===")
print(pairwise_college, "\n")

pairwise_music = pd.read_excel(linkAHP, sheet_name='Music Classes', index_col=0)
print("=== Music Classes ===")
print(pairwise_music, "\n")

pairwise_criteria = pd.read_excel(linkAHP, sheet_name='Criteria', index_col=0)
print("=== Criteria ===")
print(pairwise_criteria)


=== Learning ===
     A   B    C
A  NaN NaN  NaN
B  3.0 NaN  3.0
C  2.0 NaN  NaN 

=== Friends ===
    A   B   C
A NaN NaN NaN
B NaN NaN NaN
C NaN NaN NaN 

=== School Life ===
    A    B   C
A NaN  5.0 NaN
B NaN  NaN NaN
C NaN  5.0 NaN 

=== Vocational Training ===
    A    B    C
A NaN  9.0  7.0
B NaN  NaN  NaN
C NaN  5.0  NaN 

=== College Prep ===
     A   B    C
A  NaN NaN  NaN
B  2.0 NaN  2.0
C  NaN NaN  NaN 

=== Music Classes ===
    A    B    C
A NaN  6.0  4.0
B NaN  NaN  NaN
C NaN  3.0  NaN 

=== Criteria ===
                     Learning  Friends  School Life  Vocational Training  \
Learning                  NaN      4.0          3.0                  NaN   
Friends                   NaN      NaN          7.0                  3.0   
School Life               NaN      NaN          NaN                  NaN   
Vocational Training       NaN      NaN          5.0                  NaN   
College Prep              NaN      5.0          5.0                  NaN   
Music Classes      

### Data Prepartion

#### Transform adjacency matrixes into pairwise comparissons

In [52]:
# Build function to make the comparissons readable in the AHP library

def adjacency_to_comparisons(pairwise_df):
    """Convert adjacency matrix to comparison dictionary"""
    G = nx.from_pandas_adjacency(pairwise_df, create_using=nx.MultiDiGraph())
    return {(e[0], e[1]): e[2]['weight']
            for e in G.edges(data=True)
            if np.isfinite(e[2]['weight'])}

In [53]:
# Transform adjacency matrixes into pairwise comparisons for all criteria

learning_comparisons = adjacency_to_comparisons(pairwise_learning)
friends_comparisons = adjacency_to_comparisons(pairwise_friends)
schoollife_comparisons = adjacency_to_comparisons(pairwise_schoollife)
vocational_comparisons = adjacency_to_comparisons(pairwise_vocational)
college_comparisons = adjacency_to_comparisons(pairwise_college)
music_comparisons = adjacency_to_comparisons(pairwise_music)
criteria_comparisons = adjacency_to_comparisons(pairwise_criteria)

print("COMPARISONS EXTRACTED")
print(f"Learning comparisons: {len(learning_comparisons)} pairs")
print(f"Friends comparisons: {len(friends_comparisons)} pairs")
print(f"School Life comparisons: {len(schoollife_comparisons)} pairs")
print(f"Vocational comparisons: {len(vocational_comparisons)} pairs")
print(f"College Prep comparisons: {len(college_comparisons)} pairs")
print(f"Music comparisons: {len(music_comparisons)} pairs")
print(f"Criteria comparisons: {len(criteria_comparisons)} pairs")

COMPARISONS EXTRACTED
Learning comparisons: 3 pairs
Friends comparisons: 0 pairs
School Life comparisons: 2 pairs
Vocational comparisons: 3 pairs
College Prep comparisons: 2 pairs
Music comparisons: 3 pairs
Criteria comparisons: 12 pairs


#### **IMPORTANT**: For the rest of the analysis, I removed Friends from comparison because there is no meaningful comparison. In Dr. Saaty's video, the data shows that his son did not have a preference of friends for any school. Therefore, all the data is 1 compared to each other.

#### Dr. Saaty Video Citation: Minute 8:04
https://umamherst.instructure.com/courses/34009/files/folder/File_for_SESSIONs/Session%20-%20Optimization?preview=15382711

### AHP Installation & Performance

#### Install AHP library and verify

In [54]:
# Install ahpy

import ahpy

!pip install AHPy

#### Create compare objects

In [55]:
# Use the compare function to create compare objects

learning = ahpy.Compare('Learning', learning_comparisons, random_index='saaty')
schoollife = ahpy.Compare('School Life', schoollife_comparisons, random_index='saaty')
vocational = ahpy.Compare('Vocational Training', vocational_comparisons, random_index='saaty')
college = ahpy.Compare('College Prep', college_comparisons, random_index='saaty')
music = ahpy.Compare('Music Classes', music_comparisons, random_index='saaty')
criteria = ahpy.Compare('Criteria', criteria_comparisons, random_index='saaty')

#### Create hierarchy

In [56]:
# Create hierarchy Goal <- Criteria <- Alternatives

criteria.add_children([learning, schoollife, vocational, college, music])

print("\nHierarchy created: Goal <- Criteria <- Alternatives (Schools A, B, C)")



Hierarchy created: Goal <- Criteria <- Alternatives (Schools A, B, C)


#### Create weights

In [57]:
# Create weights to see which criteria was more valuable

print("CRITERIA WEIGHTS")
weights_df = pd.Series(criteria.global_weights).to_frame(name='Weight')
weights_df = weights_df.sort_values('Weight', ascending=False)
weights_df['Weight %'] = (weights_df['Weight'] * 100).round(2)
print(weights_df)

CRITERIA WEIGHTS
                     Weight  Weight %
Learning             0.3753     37.53
College Prep         0.2627     26.27
Music Classes        0.1273     12.73
Friends              0.1245     12.45
Vocational Training  0.0717      7.17
School Life          0.0385      3.85


#### Final school rankings

In [58]:
# Identify the best school option

print("FINAL SCHOOL RANKINGS")
priorities_df = pd.Series(criteria.target_weights).to_frame(name='Score')
priorities_df = priorities_df.sort_values('Score', ascending=False)
priorities_df['Score %'] = (priorities_df['Score'] * 100).round(2)
priorities_df['Rank'] = range(1, len(priorities_df) + 1)
print(priorities_df)

FINAL SCHOOL RANKINGS
    Score  Score %  Rank
B  0.3732    37.32     1
A  0.2854    28.54     2
C  0.2169    21.69     3


#### Access consistency

In [59]:
# The AHP algorithm assumes when you are comparing you are consistent ; but it may detect if you have been inconsistent, create a consistency assessment

print("CONSISTENCY RATIO ASSESSMENT")
print("(Values should be < 0.1 for acceptable consistency)")

assessment = [(val.name, round(val.consistency_ratio, 4))
              for val in [learning, schoollife, vocational, college, music, criteria]]
consistency_df = pd.DataFrame(assessment, columns=['Criterion', 'Consistency Ratio'])
print(consistency_df)


CONSISTENCY RATIO ASSESSMENT
(Values should be < 0.1 for acceptable consistency)
             Criterion  Consistency Ratio
0             Learning             0.0516
1          School Life             0.0000
2  Vocational Training             0.2005
3         College Prep             0.0000
4        Music Classes             0.0516
5             Criteria             0.1598


#### Review school decision summary

In [60]:
# Identify best school option

print("DECISION SUMMARY")
best_school = priorities_df.index[0]
best_score = priorities_df.iloc[0]['Score %']
print(f"\nRecommended School: {best_school}")
print(f"Overall Score: {best_score}%")
print(f"\nTop 3 Most Important Criteria:")
for i, (criterion, weight) in enumerate(weights_df.head(3).iterrows(), 1):
    print(f"  {i}. {criterion}: {weight['Weight %']}%")

DECISION SUMMARY

Recommended School: B
Overall Score: 37.32%

Top 3 Most Important Criteria:
  1. Learning: 37.53%
  2. College Prep: 26.27%
  3. Music Classes: 12.73%
